In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import fft
import pyvisa
import ipywidgets as widgets

# Function to measure noise and perform FFT
def measure_noise(smu, frequency, drain_current_range, current_amplifier_gain, gate_voltage_range, voltage_amplifier_gain):
    # Configure the SMU for current noise measurement at the current frequency
    smu.write(f":FREQ {frequency}")

    # Measure current noise
    num_points = 1000  # Number of data points to acquire
    smu.write(":TRIG:SOUR IMM")  # Immediate trigger
    current_data = smu.query_ascii_values(f":TRAC:DATA? {num_points}")  # Read the acquired current data
    current_data = np.array(current_data) * current_amplifier_gain  # Amplify the current data if necessary

    # Perform FFT to obtain the current noise spectrum
    sampling_frequency = smu.query(":FREQ?")
    current_noise_spectrum = fft(current_data)
    freq_axis = np.fft.fftfreq(num_points, 1 / float(sampling_frequency))

    # Calculate current noise power
    current_noise_power = np.mean(np.abs(current_noise_spectrum) ** 2)

    # Measure resistance noise
    voltage_data = smu.query_ascii_values(f":TRAC:DATA? {num_points}")  # Read the acquired voltage data
    voltage_data = np.array(voltage_data) * voltage_amplifier_gain  # Amplify the voltage data if necessary

    # Perform FFT to obtain the resistance noise spectrum
    voltage_noise_spectrum = fft(voltage_data)

    # Calculate resistance noise power
    resistance_noise_power = np.mean(np.abs(voltage_noise_spectrum) ** 2)

    return freq_axis, current_noise_spectrum, current_noise_power, voltage_noise_spectrum, resistance_noise_power

# Function to save data and plots
def save_data_and_plots(freq_axis, current_noise_spectrum, current_noise_power, voltage_noise_spectrum, resistance_noise_power, current_noise_filename, resistance_noise_filename):
    # Save current noise spectrum to a file
    np.savetxt(current_noise_filename, np.column_stack((freq_axis, np.abs(current_noise_spectrum))))

    # Save resistance noise spectrum to a file
    np.savetxt(resistance_noise_filename, np.column_stack((freq_axis, np.abs(voltage_noise_spectrum) ** 2)))

    # Plot the current noise spectrum
    plt.figure()
    plt.semilogx(freq_axis, np.abs(current_noise_spectrum))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Current Noise Spectrum (A/√Hz)')
    plt.grid(True)
    plt.title('Current Noise Spectrum')

    # Save the current noise plot
    plt.savefig(current_noise_filename.replace('.txt', '.png'))

    # Plot the resistance noise spectrum
    plt.figure()
    plt.semilogx(freq_axis, np.abs(voltage_noise_spectrum) ** 2)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Resistance Noise Spectrum (V²/Hz)')
    plt.grid(True)
    plt.title('Resistance Noise Spectrum')

    # Save the resistance noise plot
    plt.savefig(resistance_noise_filename.replace('.txt', '.png'))

    # Show the plots
    plt.show()

# Create widget for filenames and variables input
filename_widget = widgets.Text(value='data.txt', description='Filename:', layout={'width': '300px'})
current_noise_filename_widget = widgets.Text(value='current_noise.txt', description='Current Noise Filename:', layout={'width': '300px'})
resistance_noise_filename_widget = widgets.Text(value='resistance_noise.txt', description='Resistance Noise Filename:', layout={'width': '300px'})
drain_current_range_widget = widgets.FloatText(value=1e-5, description='Drain Current Range:', layout={'width': '300px'})
current_amplifier_gain_widget = widgets.FloatText(value=1e6, description='Current Amplifier Gain:', layout={'width': '300px'})
gate_voltage_range_widget = widgets.FloatText(value=1.0, description='Gate Voltage Range:', layout={'width': '300px'})
voltage_amplifier_gain_widget = widgets.FloatText(value=1e6, description='Voltage Amplifier Gain:', layout={'width': '300px'})

# Create button widget
button = widgets.Button(description="Save and Close")

# Define function to handle button click event
def on_button_click(button):
    # Retrieve the input values
    filename = filename_widget.value
    current_noise_filename = current_noise_filename_widget.value
    resistance_noise_filename = resistance_noise_filename_widget.value
    drain_current_range = drain_current_range_widget.value
    current_amplifier_gain = current_amplifier_gain_widget.value
    gate_voltage_range = gate_voltage_range_widget.value
    voltage_amplifier_gain = voltage_amplifier_gain_widget.value


    # Save the input values to a file
    input_values = {
        'filename': filename,
        'current_noise_filename': current_noise_filename,
        'resistance_noise_filename': resistance_noise_filename,
        'drain_current_range': drain_current_range,
        'current_amplifier_gain': current_amplifier_gain,
        'gate_voltage_range': gate_voltage_range,
        'voltage_amplifier_gain': voltage_amplifier_gain
    }
    np.save('input_values.npy', input_values)

    # Close the widget
    widget_box.close()

# Register button click event
button.on_click(on_button_click)

# Display the widget and button
widget_box = widgets.VBox([filename_widget, current_noise_filename_widget, resistance_noise_filename_widget, drain_current


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


ImportError: cannot import name 'KeysightB2902A' from 'keysight' (/opt/homebrew/lib/python3.10/site-packages/keysight/__init__.py)

In [ ]:
# Connect to the SMU
rm = pyvisa.ResourceManager()
rm.list_resources()

In [ ]:

smu = rm.open_resource('<Your SMU VISA Address>')  # Replace '<Your SMU VISA Address>' with the appropriate VISA address

# Set up SMU parameters
gate_voltage = 1.0  # Set the gate voltage value for current noise measurement

# Configure the SMU for current noise measurement
smu.write(f":VOLT {gate_voltage}")
smu.write(":SOUR:FUNC VOLT")
smu.write(":SENS:FUNC 'CURR'")
smu.write(":SENS:CURR:RANG:AUTO ON")
smu.write(":SENS:CURR:RANG {drain_current_range}")

# Perform frequency sweep and measure noise at each frequency
for freq in frequency_range:
    freq_axis, current_noise_spectrum, current_noise_power, voltage_noise_spectrum, resistance_noise_power = measure_noise(smu, freq, drain_current_range, current_amplifier_gain, gate_voltage_range, voltage_amplifier_gain)

    # Create unique filenames for each measurement
    current_noise_filename = f"current_noise_{freq}.csv"
    resistance_noise_filename = f"resistance_noise_{freq}.csv"

    # Save data and plots for the current measurement
    save_data_and_plots(freq_axis, current_noise_spectrum, current_noise_power, voltage_noise_spectrum, resistance_noise_power, current_noise_filename, resistance_noise_filename)


